# Scoring Different Prompting Techniques with DeepEval

Three different prompting techniques are evaluated in this notebook:

1. Base RAG Prompt 
2. Advanced RAG Prompt (role-based)
3. Optimized RAG Prompt (based on DSPy)

In [14]:
import pandas as pd
import deepeval
import dspy
import requests
import json
import boto3

from dotenv import load_dotenv
load_dotenv()

True

## Base/Advanced RAG Prompt

In [15]:
# Load data
with open("synthetics/testset.json", "r") as f:
	dataset = json.load(f)
print(f"Test set has {len(dataset)} samples")
print(f"Test set keys: {dataset[0].keys()}")

Test set has 12 samples
Test set keys: dict_keys(['question', 'response'])


In [16]:
# Setting constants for all the models
TOP_N = 20
MAX_TOKENS = 1024
TOP_P = 0.7
TEMPERATURE = 0.3

In [20]:
# Base Prompt
from typing import Literal

def rag_call(prompt: str, prompt_type: Literal["cite", "base"]) -> str:
    request_body = {
        "body": prompt,
        "max_tokens": 1024,
		"prompt": prompt_type,
		"top_n": TOP_N,
		"top_p": TOP_P,
        "temperature": TEMPERATURE
	}
    response = requests.post(
        url="http://greencompute-1575332443.us-east-1.elb.amazonaws.com/api/llm/rag",
        json=request_body
	)
    return response.json()["response"]

In [21]:
from tqdm.notebook import tqdm

for prompt_type in ["cite", "base"]:
	print(f"Prompt Type: {prompt_type}")
	for record in tqdm(dataset):
		generated_text = rag_call(record["question"], prompt_type)
		record[f"generated_{prompt_type}"] = generated_text

Prompt Type: cite


  0%|          | 0/12 [00:00<?, ?it/s]

Prompt Type: base


  0%|          | 0/12 [00:00<?, ?it/s]

## Optimized RAG Prompt

In [22]:
def search(query: str, top_n: int) -> list[str]:
    url = "http://greencompute-1575332443.us-east-1.elb.amazonaws.com/api/llm/retrieval"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "query": query,
        "top_n": top_n
    }

    documents = requests.post(url, headers=headers, json=data).json()["documents"]
    return [f"[{i}]" + doc["doc_title"] + doc["url"] + "\n\n" + doc["content"] for i, doc in enumerate(documents)]

class TitanLM(dspy.LM):
    def __init__(self, model: str, client, max_tokens: int = 1024, temperature: float = 0.3, top_p: float = 0.7, **kwargs):
        self.client = client
        self.history = []
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.top_p = top_p

        super().__init__(model, **kwargs)
        self.model = model
    
    def _format_message(self, prompt: str):
        body = json.dumps(
            {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": self.max_tokens,
                    "stopSequences": [],
                    "temperature": self.temperature,
                    "topP": self.top_p,
                },
            }
        )
        return body

    def generate_content(self, prompt: str) -> str:
        body = self._format_message(prompt)
        response = self.client.invoke_model(
            body=body,
            modelId=self.model,
            accept="application/json",
            contentType="application/json",
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("results")

    def __call__(self, prompt=None, messages=None, **kwargs):
        # Custom chat model working for text completion model
        prompt = '\n\n'.join([x['content'] for x in messages] + ['BEGIN RESPONSE:'])

        completions = self.generate_content(prompt)
        self.history.append({"prompt": prompt, "completions": completions})

        # Must return a list of strings
        return [completions[0].get("outputText")]

    def inspect_history(self):
        for interaction in self.history:
            print(f"Prompt: {interaction['prompt']} -> Completions: {interaction['completions']}")

In [26]:
class RAG(dspy.Module):
    def __init__(self, num_docs=20):
        self.num_docs = num_docs
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question, top_n=self.num_docs)
        return self.respond(context=context, question=question)

In [27]:
# Load the optimized RAG model
lm = TitanLM("amazon.titan-text-premier-v1:0", client=boto3.client("bedrock-runtime"))
dspy.configure(lm=lm)
rag = RAG()
rag.load("output/optimized_rag_v2.json")

In [28]:
rag("How can I increase my data center efficiency?")

Prediction(
    reasoning='There are many ways to increase data center efficiency. Some of the most common methods include improving cooling efficiency, optimizing power usage, and upgrading hardware. By implementing these strategies, data center operators can reduce energy consumption, lower costs, and improve overall performance.',
    response='To increase data center efficiency, consider improving cooling efficiency, optimizing power usage, and upgrading hardware. This can help reduce energy consumption, lower costs, and improve overall performance.'
)

In [29]:
for record in dataset:
	record["optimized_rag"] = rag(record["question"])

In [ ]:
# Save the results
import pathlib

pathlib.Path("output").mkdir(parents=True, exist_ok=True)

with open(f"output/results.json", "w") as f:
	json.dump(dataset, f)

## Evaluation

In [ ]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

# Load the results from the saved file
with open("output/results.json", "r") as f:
	results = json.load(f)

In [72]:
from typing import Literal
from deepeval.dataset import EvaluationDataset

def evaluate(prompt_type: Literal["generated_base", "generated_cite", "optimized_rag"]):
    dataset = EvaluationDataset()
    dataset.add_test_cases_from_json_file(
		file_path="output/results.json",
		input_key_name="question",
		actual_output_key_name=prompt_type,
		expected_output_key_name="response",
	)
    
    correctness_metric = GEval(
		name="Correctness",
		criteria="Determine whether the actual output is factually correct based on the expected output.",
		evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
	)

    results = dataset.evaluate([correctness_metric])

    # Save the results to a json file
    with open(f"output/{prompt_type}_metrics.json", "w") as f:
        json.dump(results.model_dump(), f)

	# Print the average of the results
    sum_results = sum([result.metrics_data[0].score for result in results.test_results])
    avg_results = sum_results / len(results.test_results)
    print(f"Average correctness score for {prompt_type}: {avg_results}")
    print(f"Saved results to output/{prompt_type}_results.json")

    return results

In [73]:
base_results = evaluate("generated_base")

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 12 test case(s) in parallel: |██████████|100% (12/12) [Time Taken: 00:04,  2.41test case/s]



Metrics Summary

  - ✅ Correctness (GEval) (score: 0.9055421913628816, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output comprehensively outlines multiple factors contributing to suboptimal energy designs in compact server facilities, such as inadequate cooling management, insufficient monitoring, and financial constraints, aligning well with the input prompt. However, the input does not specify expected output details for direct comparison, preventing a perfect score., error: None)

For test case:

  - input: Examine the underlying factors contributing to suboptimal energy designs in compact server facilities.
  - actual output: Compact server facilities often face energy efficiency challenges due to several factors. These include:

1. Inadequate cooling management: Small server rooms may not have the same cooling infrastructure as larger data centers, leading to inefficient cooling and increased energy consumption.

2. Insufficient monitoring and co

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Average correctness score for generated_base: 0.6654460537908057
Saved results to output/generated_base_results.json


In [74]:
cite_results = evaluate("generated_cite")

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 12 test case(s) in parallel: |██████████|100% (12/12) [Time Taken: 00:05,  2.36test case/s]



Metrics Summary

  - ✅ Correctness (GEval) (score: 0.8830592930671679, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output lists practical energy-saving measures like regular maintenance, air flow management, and energy-efficient equipment, aligning well with expected cost-effective strategies. However, the expected output is not provided, so assumption-based minor variance might exist., error: None)

For test case:

  - input: What simple and cost-effective energy-saving measures can small data centers implement effectively?
  - actual output: Small data centers can implement several simple and cost-effective energy-saving measures, including:

1. Regular maintenance of IT equipment and infrastructure to ensure optimal performance and energy efficiency.
2. Proper air flow management to ensure efficient cooling and reduce energy consumption.
3. Use of energy-efficient IT equipment, such as servers, storage devices, and networking equipment.
4. Implement

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Average correctness score for generated_cite: 0.883958657012261
Saved results to output/generated_cite_results.json


In [75]:
opt_results = evaluate("optimized_rag")

✨ You're running DeepEval's latest Correctness (GEval) Metric! (using gpt-4o, strict=False, async_mode=True)...

Event loop is already running. Applying nest_asyncio patch to allow async execution...


Evaluating 12 test case(s) in parallel: |██████████|100% (12/12) [Time Taken: 00:05,  2.23test case/s]



Metrics Summary

  - ✅ Correctness (GEval) (score: 0.9073008948198085, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The actual output accurately lists strategies such as adjusting variable speed fans and turning off specific units that require minimal investment for energy savings in data centers. It aligns well with the expected output, with no significant discrepancies., error: None)

For test case:

  - input: Which energy efficiency strategies, requiring minimal investment, yield 20-40% savings for small data centers?
  - actual output: Adjusting variable speed fans, turning off CRAC/ACU/CRAH/AHUs, and monitoring the effectiveness of air-based cooling are some energy efficiency strategies that require minimal investment and can yield 20-40% savings for small data centers.
  - expected output: Energy efficiency strategies for small data centers that require minimal investment and can yield 20-40% savings include turning off unused equipment, optimizing airflow 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI. 
‼️  Friendly reminder 😇: You can also run evaluations with ALL of deepeval's metrics directly on Confident AI 
instead.

Average correctness score for optimized_rag: 0.9213896297589544
Saved results to output/optimized_rag_results.json
